In [0]:
import random
from sklearn.metrics import f1_score
from statistics import mean, stdev
import math
from scipy.stats import t

def permutation_test(y_true, y_pred_baseline, y_pred_model):
  alpha = 0.05
  num_permutations = 10000
  
  array1 = []; array2 = [];
  razlike = []
  if len(y_pred_baseline) != len(y_pred_model):
    print("Nisu iste dimenzije!")
    return
  f1_before_base = f1_score(y_true, y_pred_baseline, average='micro')
  print("baseline: " + str(f1_before_base))
  f1_before_model = f1_score(y_true, y_pred_model, average='micro')
  raz_before = abs(f1_before_base - f1_before_model)
  count = 0
  for i in range(num_permutations):
    array1 = []; array2 = [];
    for j in range(len(y_pred_model)):
      pick = random.uniform(0,1)
      if pick <= alpha:
        array1.append(float(y_pred_baseline[j]))
        array2.append(float(y_pred_model[j]))
      else:
        array1.append(float(y_pred_model[j]))
        array2.append(float(y_pred_baseline[j]))
    f11 = f1_score(y_true, array1, average='micro')
    f12 = f1_score(y_true, array2, average='micro')
    raz = abs(f11 - f12)
    if raz < raz_before:
      count += 1
  # izracunaj t-statistiku
  p_value = 1 - float(count/num_permutations)
  print(p_value)
  
  if p_value < alpha:
    print("Nije slucajno! Bolji je model nego baseline. p_value = %f" % (p_value))
  else:
    print("Slucajno je bolji model od baseline-a! p_value = %f" % (p_value))
    
    


In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm

colnames = ['ID','Gene','Variation','Class']
# TEST
y_test = pd.read_csv('test_variants', header=None, sep=',', names = colnames, engine='python')
ne_zelim = np.load('na indices test.npy')
y_test = y_test[~ne_zelim]
y_test = y_test.Class.tolist()
y_test = y_test[1:]
y_test = [int(i) for i in y_test]

colnames = ['baseline_train']
base_test = pd.read_csv('test_baseline.csv', names=colnames)
base_test = base_test.baseline_train.tolist()
base_test = base_test[2:]
base_test = [int(i) for i in base_test]
#base_test = list(map(lambda x: x + 1, base_test))
  
colnames = ['logistic_test','decision_tree_test','linear_svm_test','random_forest_test','all_test','svc_test','gb_test']
data = pd.read_csv('test_word2vec.csv', names=colnames)
print('----TEST WORD2VEC---')
for col in colnames:
  y_model = data[col].tolist()[1:]
  y_model = [int(i) for i in y_model]
  print("Testing model: " + col)
  y_model = list(map(lambda x: x + 1, y_model))
  print(y_model)
  permutation_test(y_test, base_test, y_model) 

#doc2vec embeddings
colnames = ['logistic_test','decision_tree_test','linear_svm_test','random_forest_test','all_test','svc_test','gb_test']
data = pd.read_csv('test_doc2vec.csv', names=colnames)
print('----TEST DOC2VEC---')
for col in colnames:
  y_model = data[col].tolist()[1:] 
  y_model = [int(i) for i in y_model]
  print("Testing model: " + col)
  y_model = list(map(lambda x: x + 1, y_model))
  permutation_test(y_test, base_test, y_model)
  
#worddoc2vec
colnames = ['logistic_test','decision_tree_test','linear_svm_test','random_forest_test','all_test','svc_test','gb_test']
data = pd.read_csv('test_WordDoc2vec.csv', names=colnames)
print('----TEST WORDDOC2VEC---')
for col in colnames:
  y_model = data[col].tolist()[1:]
  y_model = [int(i) for i in y_model]
  print("Testing model: " + col)
  y_model = list(map(lambda x: x + 1, y_model))
  permutation_test(y_test, base_test, y_model)

In [0]:

pred1 = '6 9 7 1 7 1 1 2 1 1 1 1 9 9 7 7 2 7 7 7 7 7 7 4 2 7 7 4 2 7 2 2 7 5 2 7 1 1 1 5 1 4 1 4 7 1 6 1 1 5 4 1 6 7 4 1 4 1 4 1 4 1 4 6 4 4 2 4 1 4 4 4 4 4 4 2 5 7 7 2 7 2 7 7 7 7 7 1 1 6 2 2 2 4 2 7 2 3 2 7 7 7 1 1 7 7 7 1 3 4 1 1 1 1 1 2 2 1 1 7 4 4 7 7 7 9 1 1 7 6 7 5 2 7 1 2 1 1 5 7 7 4 5 6 6 4 1 7 7 7 2 5 5 1 5 5 4 4 1 1 2 7 2 1 7 7 2 7 3 3 7 7 7 1 1 1 1 1 4 4 4 7 3 7 7 7 5 4 1 1 1 4 5 1 4 4 7 7 7 2 1 1 7 1 7 7 7 7 7 4 7 7 7 7 7 1 7 7 7 6 7 6 1 7 2 2 4 1 6 2 4 4 1 7 1 1 3 4 4 4 4 4 4 4 4 4 4 1 4 4 5 1 1 7 1 6 6 6 7 4 4 2 2 2 4 1 1 5 6 4 5 1 1 1 1 5 1 1 5 5 5 3 5 1 1 5 5 1 5 1 5 1 5 6 5 2 2 2 5 9 3 2 6 2 2 7 4 6 5 6 1 6 6 1 1 6 1 6 5 1 7 7 2 4 7 4 2 7 2 2 2 7 7 7 7 7 2 9 2 2 7 2 1 1 7 2 5 4 4 2 1 7 7 7 1 7 1 7 6 4 7 6 2 2 4 1'
pred1 = pred1.split(' ')
pred1 = [int(i) for i in pred1]
pred2 = '6 4 4 1 4 1 4 2 1 2 1 4 4 9 2 7 7 7 7 7 7 7 7 7 2 7 7 4 2 7 2 7 7 5 2 7 1 4 1 4 1 4 1 4 7 7 6 1 1 4 4 1 6 4 4 1 4 1 1 1 1 1 1 6 4 4 4 2 4 4 4 4 4 4 4 4 5 7 7 2 7 2 7 7 7 7 7 1 1 1 2 2 7 7 7 7 2 3 2 7 7 2 2 1 7 7 1 1 3 4 1 1 1 1 1 1 7 1 4 4 7 4 7 7 7 9 1 7 7 6 7 5 7 7 1 2 7 1 5 2 7 7 5 6 6 1 7 7 7 7 2 7 5 7 5 5 4 4 1 1 7 7 2 1 7 4 7 7 7 3 7 7 7 1 1 1 7 2 4 4 4 7 3 7 7 1 5 4 1 1 2 4 5 1 1 1 6 1 7 1 1 1 2 1 7 7 7 7 7 1 7 7 7 4 4 1 7 7 7 6 7 6 7 7 1 7 1 5 1 4 2 7 1 7 7 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 2 1 1 7 1 6 6 6 4 4 1 7 7 7 4 1 1 1 4 4 5 1 1 1 1 5 1 1 5 5 5 3 5 1 1 4 4 1 5 1 5 1 5 6 5 2 2 7 2 7 5 7 2 7 2 7 1 6 5 6 1 6 6 1 4 6 1 6 5 1 7 7 2 7 7 1 7 7 2 2 2 7 7 7 7 2 2 9 2 2 7 7 7 1 7 7 5 1 7 7 1 2 2 7 1 2 1 7 7 7 7 7 2 1 4 1'
pred2 = pred2.split(' ')
pred2 = [int(i) for i in pred2]
pred3 = '7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7'
pred3 = pred3.split(' ')
pred3 = [int(i) for i in pred3]
pred4 = '4 4 4 1 4 1 1 7 1 2 1 4 9 9 7 7 7 7 7 7 7 7 7 7 2 7 7 4 2 7 7 4 7 5 2 7 1 1 1 1 1 4 1 4 1 1 1 1 1 4 4 1 1 4 4 1 4 1 1 4 1 1 1 6 4 4 4 4 1 4 4 4 4 4 4 1 5 7 7 2 2 2 7 7 7 7 7 1 1 1 2 2 2 7 7 7 7 3 7 7 7 7 2 1 7 7 7 1 4 4 1 1 1 1 1 7 7 1 7 1 4 4 7 7 7 9 1 7 7 6 7 5 2 7 1 2 4 4 7 2 7 7 4 6 6 4 7 7 7 7 2 7 5 2 5 5 4 4 4 1 7 7 7 1 7 7 7 7 3 3 7 7 7 1 4 1 4 4 4 4 4 7 3 7 7 4 5 4 1 4 4 5 5 9 1 4 6 7 7 1 1 1 2 1 7 7 7 7 7 1 7 7 7 7 1 1 7 7 7 6 7 6 7 7 4 7 4 6 6 1 1 4 1 7 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 8 1 1 7 1 6 6 6 7 4 4 2 7 7 4 1 5 5 4 1 5 1 5 1 5 5 1 1 5 5 5 3 5 5 1 5 4 1 5 1 4 5 5 6 5 7 2 2 7 7 7 7 7 7 2 7 1 6 5 6 1 6 6 1 1 6 1 6 5 5 1 7 1 7 7 7 7 7 2 2 2 7 7 7 7 7 2 9 2 2 7 2 1 3 7 7 5 2 7 7 1 7 7 1 1 7 1 7 7 7 7 2 2 2 4 4'
pred4 = pred4.split(' ')
pred4 = [int(i) for i in pred4]
pred5 = '4 4 1 1 4 1 1 7 1 7 1 4 9 9 7 7 7 7 7 7 7 7 7 7 2 7 7 7 7 7 7 4 7 1 2 7 1 1 4 1 1 1 1 4 1 1 1 1 1 4 4 1 1 4 4 1 4 1 1 1 1 1 1 1 4 4 4 4 1 4 4 4 4 4 4 1 7 7 7 2 2 7 7 7 7 7 7 1 1 1 2 2 2 7 7 7 7 7 7 7 7 7 2 1 7 7 7 4 4 4 1 1 1 1 1 7 7 1 4 1 4 4 7 7 7 9 1 7 7 6 7 7 2 7 1 2 4 1 4 7 7 7 4 6 6 4 7 7 7 7 2 7 7 7 7 7 4 4 4 7 7 7 7 7 7 7 7 7 7 7 7 7 7 1 4 1 4 4 4 4 7 7 7 7 7 4 5 4 1 7 4 5 4 9 1 4 7 7 7 1 1 1 2 1 7 7 7 7 7 1 7 7 7 7 1 1 7 7 7 7 7 7 7 7 7 7 4 1 1 1 1 1 1 7 4 7 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 4 7 7 6 6 6 7 4 4 7 7 7 7 1 5 5 4 1 5 1 5 1 5 5 1 1 5 5 5 3 5 5 1 5 5 4 5 1 4 5 5 6 5 7 7 7 7 7 7 7 7 7 2 7 1 6 5 6 1 6 6 1 1 6 1 6 5 6 1 7 1 7 7 7 7 2 2 2 2 7 7 7 7 7 2 7 2 2 7 2 1 1 7 7 7 7 7 7 1 7 7 2 1 7 7 7 7 7 7 7 2 4 4 4'
pred5 = pred5.split(' ')
pred5 = [int(i) for i in pred5]
pred6 = '6 4 1 1 4 1 1 7 4 7 1 4 9 9 2 7 2 7 7 7 7 7 7 7 2 7 7 7 2 7 2 1 7 5 2 7 1 4 1 1 1 4 1 4 1 4 6 1 1 1 4 1 6 4 4 1 4 1 1 1 4 1 4 6 4 4 4 4 1 4 4 4 4 4 4 1 5 7 7 7 7 2 7 7 7 7 7 1 1 4 2 2 2 7 7 7 2 3 2 7 7 2 1 1 7 7 7 1 3 4 1 1 1 1 1 4 7 1 4 1 4 4 7 7 7 9 1 7 7 6 7 5 7 7 7 2 7 1 7 2 7 2 5 6 6 1 7 7 7 7 2 5 5 7 5 5 4 4 1 1 7 7 7 1 7 7 7 7 7 3 7 7 7 1 1 1 4 1 4 4 4 7 3 7 7 1 5 4 1 1 4 4 5 9 1 4 1 1 7 2 1 1 4 1 7 7 7 7 7 1 7 7 7 7 1 1 7 7 7 6 7 6 7 7 4 7 1 1 6 1 7 8 1 7 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 8 1 1 7 7 6 6 6 7 4 4 7 7 7 4 4 5 5 4 4 5 1 5 1 4 5 1 1 5 5 5 3 5 5 1 4 5 1 5 1 5 5 5 6 5 7 2 7 2 2 7 7 7 7 7 7 1 6 5 6 1 6 6 1 4 6 1 6 5 1 1 7 1 2 7 7 7 7 2 2 2 7 7 7 7 2 2 9 2 2 7 7 2 3 7 7 5 2 2 7 1 7 7 2 1 7 7 7 7 7 7 7 2 2 4 1'
pred6 = pred6.split(' ')
pred6 = [int(i) for i in pred6]

permutation_test(y_test, base_test, pred1)
permutation_test(y_test, base_test, pred2)
permutation_test(y_test, base_test, pred3)
permutation_test(y_test, base_test, pred4)
permutation_test(y_test, base_test, pred5)
permutation_test(y_test, base_test, pred6)

In [0]:
import pandas as pd
import numpy as np
colnames = ['ID','Gene','Variation','Class']
# TEST
y_test = pd.read_csv('test_variants', header=None, sep=',', names = colnames, engine='python')
ne_zelim = np.load('na indices test.npy')
y_test = y_test[~ne_zelim]
y_test = y_test.Class.tolist()
y_test = y_test[1:]
y_test = [int(i) for i in y_test]

colnames = ['baseline_train']
base_test = pd.read_csv('test_baseline.csv', names=colnames)
base_test = base_test.baseline_train.tolist()
base_test = base_test[2:]
base_test = [int(i) for i in base_test]
#print(base_test)
#base_test = list(map(lambda x: x + 1, base_test))



pred = '6 4 4 1 4 1 1 2 1 7 1 7 9 9 2 7 7 7 7 7 7 7 7 7 2 7 7 4 2 7 4 4 2 1 2 7 1 1 1 1 1 4 1 4 1 1 6 1 1 1 4 1 6 4 4 1 4 1 1 1 1 1 1 6 4 4 4 4 1 4 4 4 4 4 4 1 5 7 7 2 2 2 7 7 7 7 7 1 1 1 2 2 2 4 7 7 2 3 2 7 7 7 2 1 7 7 7 4 3 4 1 1 1 1 1 1 7 1 1 7 4 4 7 7 7 9 1 7 7 6 7 5 2 7 1 2 4 1 7 2 7 7 1 6 6 4 7 7 7 7 2 7 5 7 5 5 4 4 1 1 7 7 2 1 7 7 7 7 7 3 7 7 7 1 1 1 4 4 4 4 4 7 3 7 7 1 5 4 1 7 4 4 5 4 1 4 1 7 7 1 1 1 1 1 7 7 7 7 7 1 7 7 7 7 1 1 7 7 7 6 7 6 7 7 1 7 1 6 6 1 4 2 1 7 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 1 7 7 6 6 6 7 4 4 7 7 7 4 1 1 4 4 1 5 1 1 1 1 5 1 1 5 5 5 3 5 1 1 4 4 1 5 1 5 1 5 6 5 7 2 2 2 7 7 7 7 7 2 7 1 6 5 6 1 6 6 1 1 6 1 6 5 1 7 7 1 1 7 4 7 2 2 2 2 7 7 7 7 7 2 9 2 2 7 1 4 1 7 7 5 4 7 7 1 7 7 2 1 2 7 7 1 7 7 2 2 1 4 1'
pred = pred.split(' ')
pred = [int(i) for i in pred]
permutation_test(y_test, base_test, pred)
